In [1]:
import jupyter_manim

In [ ]:
%%manim SineLaw
from manimlib.scene.scene import Scene
from manimlib.imports import *

class ArcBetweenVectors(Arc):
    def __init__(self, radius, d1, d2, center, invert_angle=False,**kwargs):
        line1 = Line(center.get_center(),d1.get_center())
        line2 = Line(center.get_center(),d2.get_center())
        h = Line(center.get_center(),center.get_center()+RIGHT)
        angle = angle_between_vectors(line1.get_unit_vector(),line2.get_unit_vector())
        h1 = angle_between_vectors(h.get_unit_vector(),line1.get_unit_vector())
        h2 = angle_between_vectors(h.get_unit_vector(),line2.get_unit_vector())
        if line1.get_angle() <= line2.get_angle():
            start_angle = h1
        else:
            start_angle = h2
        if invert_angle:
            start_angle = -start_angle
        super().__init__(start_angle, angle,radius=radius,arc_center=center.get_center(), **kwargs)

    def get_angle(self):
        return self.angle
    
class LabelFromArc(TexMobject):
    CONFIG = {
        "distance_proportion": 1.2
    }
    def __init__(self, arc, tex_height, *tex_strings, **kwargs):
        super().__init__(*tex_strings, **kwargs)
        self.set_height(tex_height)
        center = arc.get_arc_center()
        max_size = max(self.get_width(),self.get_height()) * self.distance_proportion/ 2
        vector = Line(center,arc.point_from_proportion(0.5)).get_vector()
        end_coord = center+vector + normalize(vector)*max_size
        self.move_to(end_coord)
        

    
class Polygon(Polygon):
    def get_center_of_edges(self,buff=SMALL_BUFF*3):
        vertices = self.get_vertices()
        coords_vertices = []
        for i in range(len(vertices)):
            if i < len(vertices)-1:
                p1,p2 = [vertices[i],vertices[i+1]]
            else:
                p1,p2 = [vertices[-1],vertices[0]]
            guide_line = Line(p1,p2)
            side = guide_line.get_center()
            normal_direction = guide_line.copy()
            normal_direction.rotate(-PI/2)
            vector_normal_direction = normal_direction.get_unit_vector()
            direction = Dot(side).shift(vector_normal_direction*buff).get_center()
            coords_vertices.append(direction)

        return coords_vertices

class SineLaw(Scene):
    CONFIG = {
        "triangle_config": {
            "color": RED,
            "stroke_width": 8,
        },
        "tex_map": {
            "tex_to_color_map": {
                "\\alpha": RED_A, 
                "\\beta": TEAL_A,
                "\\gamma": PURPLE_A,
                "A": RED_A, 
                "B": TEAL_A,
                "C": PURPLE_A,
                "x": GREEN_A,
                "h_1": YELLOW_B,
                "h_2": BLUE_B,
            }
        }
    }
    def construct(self):
        du = UP*1.5
        d1 = Dot(LEFT*4+du)
        d2 = Dot(RIGHT*2+du)
        d3 = Dot(RIGHT*4+UP*2+du)
        triangle = Polygon(
            d1.get_center(),d2.get_center(),d3.get_center(),**self.triangle_config
        )
        def frac_string(n,d):
            return ["{",n,"\\over",d,"}"]
        def frac_strings(n,d):
            return ["{",*n,"\\over",*d,"}"]
        sina_t = ["{\\rm sin}","\\alpha"]
        sinb_t = ["{\\rm sin}","\\beta"]
        sinc_t = ["{\\rm sin}","\\gamma"]
        cosa_t = ["{\\rm cos}","\\alpha"]
        cosb_t = ["{\\rm cos}","\\beta"]
        cosc_t = ["{\\rm cos}","\\gamma"]
        formulas_sine_string_1 = [
            [*sinb_t,"=",*frac_string("h_1","C")],
            [*sinc_t,"=",*frac_string("h_1","B")],
            ["C","\\,",*sinb_t,"=","h_1"],
            ["B","\\,",*sinc_t,"=","h_1"],
            ["C","\\,",*sinb_t,"=","B","\\,",*sinc_t],
            [*frac_strings(["C"],sinc_t),"=",*frac_strings(["B"],sinb_t)]
        ]
        formulas_sine_string_2 = [
            [*sina_t,"=",*frac_string("h_2","B")],
            [*sinb_t,"=",*frac_string("h_2","A")],
            ["B","\\,",*sina_t,"=","h_2"],
            ["A","\\,",*sinb_t,"=","h_2"],
            ["B","\\,",*sina_t,"=","A","\\,",*sinb_t],
            [*frac_strings(["B"],sinb_t),"=",*frac_strings(["A"],sina_t)]
        ]
        sine_law = TexMobject(*[
            *frac_strings(["C"],sinc_t),"=",*frac_strings(["B"],sinb_t),"=",*frac_strings(["A"],sina_t),
        ],**self.tex_map).scale(0.9)
        formulas_sine_1 = VGroup(*[
            TexMobject(*f,**self.tex_map) for f in formulas_sine_string_1
        ])
        # formulas_sine.arrange_in_grid(None,2)
        formulas_sine_arrange_1 = VGroup(
            formulas_sine_1[:2].arrange(RIGHT,buff=1),
            formulas_sine_1[2:4].arrange(RIGHT,buff=1),
            formulas_sine_1[4:].arrange(DOWN),
        ).arrange(DOWN,buff=0.7).scale(0.9)
        formulas_sine_2 = VGroup(*[
            TexMobject(*f,**self.tex_map) for f in formulas_sine_string_2
        ])
        # formulas_sine.arrange_in_grid(None,2)
        formulas_sine_arrange_2 = VGroup(
            formulas_sine_2[:2].arrange(RIGHT,buff=1),
            formulas_sine_2[2:4].arrange(RIGHT,buff=1),
            formulas_sine_2[4:].arrange(DOWN),
        ).arrange(DOWN,buff=0.7).scale(0.9)
        formulas_sine_arrange_1.to_edge(DOWN,buff=0.3)
        formulas_sine_arrange_1.to_edge(LEFT,buff=1)
        formulas_sine_arrange_2.to_edge(DOWN,buff=0.3)
        formulas_sine_arrange_2.to_edge(RIGHT,buff=1)
        sine_law.align_to(formulas_sine_arrange_1,DOWN)
        triangle.set_x(0)
        center_vertices = triangle.get_center_of_edges()
        labels = VGroup(*[
            TexMobject(label,**self.tex_map).move_to(point) for label,point in zip(["C","B","A"],center_vertices)
        ])
        fs1 = formulas_sine_1
        fs2 = formulas_sine_2
        # ------------------------------
        h1 = TexMobject("h_1",**self.tex_map)
        h2 = TexMobject("h_2",**self.tex_map)
        x = TexMobject("x",**self.tex_map)
        h1_line = self.get_h(d2,d1,d3)
        h2_line = DashedLine(d3.get_center()+RIGHT*0.09,[d3.get_x()+0.09,d2.get_y()-0.09,0])
        h3_line = DashedLine(d2.get_center()+RIGHT*0.09,h2_line.get_end())
        rec_1 = Square().set_width(0.25)
        rec_1 = VMobject().set_points_as_corners([rec_1.get_corner(v) for v in [UR,UL,DL]])
        rec_2 = rec_1.deepcopy()
        rec_1.next_to(h2_line.get_end(),UL,buff=0)
        rec_2.rotate(h1_line.get_angle())
        rec_2.next_to(h1_line.get_end(),DL,buff=0)
        rec_2.shift(DOWN*0.1+RIGHT*0.05)
        x.next_to(h3_line,DOWN,0.1)
        h1.next_to(h1_line,RIGHT,0.1)
        h1.shift(LEFT*0.15)
        h2.next_to(h2_line,RIGHT,0.1)
        # h2_line.rotate(PI,about_point=h2_line.get_start())
        # ------------------------------
        alpha_arc = ArcBetweenVectors(0.3,d1,d3,d2)
        beta_arc = ArcBetweenVectors(1.7,d2,d3,d1)
        gamma_arc = ArcBetweenVectors(1,d1,d2,d3)
        alpha_p_arc = ArcBetweenVectors(0.4,Dot(h2_line.get_end()),d3,d2)
        gamma_arc.rotate(gamma_arc.get_angle()*0.9,about_point=gamma_arc.get_arc_center())
        alpha = LabelFromArc(alpha_arc,labels[0].get_width()*0.7,"\\alpha",distance_proportion=1.9,**self.tex_map)
        beta = LabelFromArc(beta_arc,labels[0].get_width()*1.1,"\\beta",distance_proportion=1.9,**self.tex_map)
        gamma = LabelFromArc(gamma_arc,labels[0].get_width()*1.1,"\\gamma",distance_proportion=1.9,**self.tex_map)
        alpha_p = LabelFromArc(alpha_p_arc,labels[0].get_width()*1.1,"\\alpha'",distance_proportion=1.9,**self.tex_map)
        alpha.shift(LEFT*0.25+DOWN*0.1)
        but = TexMobject("{\\rm sin}(\\pi-\\alpha)={\\rm sin}(\\alpha)",**self.tex_map)
        but.to_corner(UL)
        t1 = Polygon(
            d1.get_center(),d2.get_center(),h1_line.get_end(),
            color=ORANGE,stroke_width=0,fill_opacity=0
        )
        t2 = Polygon(
            d2.get_center(),d3.get_center(),h1_line.get_end(),
            color=ORANGE,stroke_width=0,fill_opacity=0
        )
        t3 = Polygon(
            d2.get_center(),h3_line.get_end(),h2_line.get_start(),
            color=ORANGE,stroke_width=0,fill_opacity=0
        )
        t4 = Polygon(
            d1.get_center(),h3_line.get_end(),h2_line.get_start(),
            color=ORANGE,stroke_width=0,fill_opacity=0
        )
        def show_triange(t):
            t.set_fill(None,0.3)
            return t
        def hide_triange(t):
            t.set_fill(None,0)
            return t
        self.add(t1,t2,t3,t4)
        # - SHOW CREATIONS
        self.add_foreground_mobject(triangle)
        self.play(
            ShowCreation(triangle,rate_func=linear),
            LaggedStart(*list(map(Write,labels)),lag_ratio=0.8),
            run_time=2.5
        )
        self.wait()
        self.play(
            LaggedStart(*[
                TransformFromCopy(m1,m2)
                for m1,m2 in zip(labels[::-1],[alpha,beta,gamma])
            ],lag_ratio=0.7),
            LaggedStart(*list(map(ShowCreation,[alpha_arc,beta_arc,gamma_arc])),lag_ratio=0.7),
            run_time=3.5
        )
        self.wait()
        self.play(LaggedStart(*list(map(Write,[h1_line,h1,rec_2])),lag_ratio=0.5))
        # - TRANSFORMATIONS
        C,B,A = labels
        self.play(ApplyFunction(show_triange,t1))
        self.wait()
        self.play(
            LaggedStart(
                TransformFromCopy(beta[0],fs1[0][1]),
                TransformFromCopy(h1[0],fs1[0][-4]),
                TransformFromCopy(C[0],fs1[0][-2]),
                lag_ratio=0.7
            ),
            LaggedStart(*[Write(fs1[0][i]) for i in [0,2,-3]]),
            run_time=5
        )
        self.wait()
        self.play(ApplyFunction(hide_triange,t1))
        self.wait()
        self.play(ApplyFunction(show_triange,t2))
        self.wait()
        self.play(
            LaggedStart(
                TransformFromCopy(gamma[0],fs1[1][1]),
                TransformFromCopy(h1[0],fs1[1][-4]),
                TransformFromCopy(B[0],fs1[1][-2]),
                lag_ratio=0.7
            ),
            LaggedStart(*[Write(fs1[1][i]) for i in [0,2,-3]]),
            run_time=5
        )
        self.wait()
        self.play(ApplyFunction(hide_triange,t2))
        #  - - - - - - - -
        self.wait()
        self.play(
            LaggedStart(
                TransformFromCopy(fs1[0][-2],fs1[2][0]),
                AnimationGroup(
                    TransformFromCopy(fs1[0][0],fs1[2][2]),
                    TransformFromCopy(fs1[0][1],fs1[2][3]),
                    lag_ratio=0
                ),
                TransformFromCopy(fs1[0][2],fs1[2][4]),
                TransformFromCopy(fs1[0][-4],fs1[2][-1]),
                lag_ratio=0.3
            ),
            # LaggedStart(*[Write(fs1[1][i]) for i in [0,2,-3]]),
            run_time=5
        )
        self.wait()
        self.play(
            LaggedStart(
                TransformFromCopy(fs1[1][-2],fs1[3][0]),
                AnimationGroup(
                    TransformFromCopy(fs1[1][0],fs1[3][2]),
                    TransformFromCopy(fs1[1][1],fs1[3][3]),
                    lag_ratio=0
                ),
                TransformFromCopy(fs1[1][2],fs1[3][4]),
                TransformFromCopy(fs1[1][-4],fs1[3][-1]),
                lag_ratio=0.3
            ),
            # LaggedStart(*[Write(fs1[1][i]) for i in [0,2,-3]]),
            run_time=5
        )
        self.wait()
        self.play(
            TransformFromCopy(fs1[2][:4],fs1[4][:4]),
            TransformFromCopy(fs1[3][:4],fs1[4][-4:]),
            Write(fs1[4][4]),
            run_time=5
        )
        self.wait()
        self.play(
            LaggedStart(
                TransformFromCopy(fs1[4][0],fs1[5][1]),
                TransformFromCopy(fs1[4][-2:],fs1[5][3:5]),
                TransformFromCopy(fs1[4][-4],fs1[5][-5]),
                TransformFromCopy(fs1[4][2:4],fs1[5][-3:-1]),
                lag_ratio=0.5
            ),
            # TransformFromCopy(fs1[4][-2:],fs1[5][3:5]),
            # TransformFromCopy(fs1[3][:4],fs1[4][-4:]),
            LaggedStart(
                Write(fs1[5][2]),
                Write(fs1[5][-4]),
                Write(fs1[5][6]),
                lag_ratio=0.5
            ),
            run_time=5
        )
        self.wait()
        # ------------------------------
        self.play(LaggedStart(*list(map(Write,[h2_line,h2,h3_line,x,rec_1])),lag_ratio=0.5))
        self.wait()
        self.play(Write(alpha_p),Write(alpha_p_arc))
        self.wait()
        self.play(Write(but))
        self.wait()
        self.play(Indicate(but),Indicate(alpha_p),Indicate(alpha_p_arc),run_time=3)
        self.wait()
        self.play(ApplyFunction(show_triange,t3))
        self.wait()
        self.play(
            LaggedStart(
                TransformFromCopy(alpha_p[0],fs2[0][1]),
                TransformFromCopy(h2[0],fs2[0][-4]),
                TransformFromCopy(B[0],fs2[0][-2]),
                lag_ratio=0.7
            ),
            LaggedStart(*[Write(fs2[0][i]) for i in [0,2,-3]]),
            run_time=5
        )
        self.wait()
        self.play(ApplyFunction(hide_triange,t3))
        self.wait()
        self.play(ApplyFunction(show_triange,t4))
        self.wait()
        self.play(
            LaggedStart(
                TransformFromCopy(beta[0],fs2[1][1]),
                TransformFromCopy(h2[0],fs2[1][-4]),
                TransformFromCopy(A[0],fs2[1][-2]),
                lag_ratio=0.7
            ),
            LaggedStart(*[Write(fs2[1][i]) for i in [0,2,-3]]),
            run_time=5
        )
        self.wait()
        self.play(ApplyFunction(hide_triange,t4))
        # ------------------------------
        self.play(
            LaggedStart(*[FadeIn(f) for f in fs2[2:]],lag_ratio=0.5),
            run_time=8
        )
        self.wait()
        # self.add(sine_law)
        self.play(
            ReplacementTransform(fs1[-1],sine_law[:len(fs1[-1])]),
            ReplacementTransform(fs2[-1],sine_law[-len(fs2[-1]):]),
            run_time=2.5
        )
        sine_law.save_state()
        self.wait()
        self.play(
            Succession(
                FadeToColor(sine_law,YELLOW),
                Restore(sine_law)
            ),
            AnimationGroup(
                ShowCreationThenDestructionAround(sine_law.copy()),
                ShowCreationThenDestructionAround(sine_law.copy()),
                lag_ratio=1
            )
        )
        # self.add(but,h1_line,h2_line,h3_line,rec_1,rec_2,x,h1,h2,alpha_p_arc,alpha_p)
        # self.add(labels,alpha,beta,gamma)
        # self.add(alpha_arc,beta_arc,gamma_arc)
        # self.add(triangle,formulas_sine_1,formulas_sine_2,sine_law)
        # self.add_foreground_mobject(triangle)
        self.wait()
        
    def get_h(self, dot, d1, d2,invert=True):
        line = Line(d1.get_center(),d2.get_center())
        vector = line.get_unit_vector()
        sign = 1 if invert else -1
        normal_vector = rotate_vector(vector,sign*PI/2)
        def get_distance_point_line(line,dot):
            x_0, y_0, z_0 = dot.get_center()
            X_0 = line.point_from_proportion(0)
            X_1 = line.point_from_proportion(1)
            x_1, y_1, z_1 = X_0
            x_2, y_2, z_2 = X_1
            return abs((x_2-x_1)*(y_1-y_0)-(x_1-x_0)*(y_2-y_1)/get_norm(line.get_vector()))
        distance = get_distance_point_line(line,dot)
        return DashedLine(dot.get_center(),dot.get_center()+distance*normal_vector)
